In [1]:
from datetime import datetime
import pandas as pd
import feather
import os
import marlin
from marlin.marlin_service_pb2 import DataType, TransformJobType, TransformOutputStores
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page

In [2]:
pip install marlinfs==0.0.1.16rc1

Note: you may need to restart the kernel to use updated packages.


In [3]:
df1 = pd.read_feather('data/demo_data.feather')
df1.head(5)

,cust_id,age,job,marital,education
0,100,56,housemaid,married,basic.4y
1,101,57,services,married,high.school
2,102,37,services,married,high.school
3,103,40,admin.,married,basic.6y
4,104,56,services,married,high.school


In [4]:
event_timestamp = int(datetime.timestamp(datetime(2020,4,30))) # 04/30/2020 @ 12:00am (UTC)
df1['event_timestamp']=event_timestamp

In [5]:
df1.head(1)

,cust_id,age,job,marital,education,event_timestamp
0,100,56,housemaid,married,basic.4y,1588230000


In [6]:
name='transform_1'
version='1'
entities={'cust_id':DataType.LONG}
features={'age': DataType.LONG, 
          'job': DataType.STRING,
          'marital': DataType.STRING, 
          'education': DataType.STRING}
output_stores=[TransformOutputStores.BATCH_STORE, TransformOutputStores.ONLINE_STORE]

In [7]:
client = marlin.client().batch_ingestion_client(name,version,entities,features,output_stores)

In [10]:
@client.transform_function
def process():
    return df1

In [11]:
process()

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1606762029.102559000","description":"Error received from peer ipv4:34.213.53.17:443","file":"src/core/lib/surface/call.cc","file_line":1062,"grpc_message":"Socket closed","grpc_status":14}"
>

In [8]:
exploration_client = marlin.client().exploration_client()

In [9]:
exploration_client.list_all_transform()

transform_key {
  transform_name: "transform_1"
  version: "1"
}

In [10]:
exploration_client.list_partitions('transform_1', '1')

['2020-11-30-06', '2020-11-30-07', '2020-11-30-08', '2020-11-30-18']

In [11]:
exploration_client.read_between_partitions('transform_1', '1', '2020-11-30-07', '2020-11-30-08')

,ingestion_time,event_time,cust_id,age,education,job,marital
0,1606723061704,1588230000,100,56,basic.4y,housemaid,married
1,1606723062208,1588230000,102,37,high.school,services,married
2,1606723062208,1588230000,103,40,basic.6y,admin.,married
3,1606723062208,1588230000,104,56,high.school,services,married
4,1606723062208,1588230000,105,45,basic.9y,services,married
...,...,...,...,...,...,...,...
9992,1606724444441,1588230000,36030,46,university.degree,admin.,single
9993,1606724444441,1588230000,36029,37,professional.course,blue-collar,single
9994,1606724444441,1588230000,36028,42,university.degree,entrepreneur,married
9995,1606724444441,1588230000,36027,62,university.degree,retired,single


In [12]:
exploration_client.transform_dependency_graph_str('transform_1', '1')

'+--- transform_1:1'